<a href="https://colab.research.google.com/github/cristianmejia00/clustering/blob/main/Topic_Models_using_BERTopic_TOPIC_MODEL_20241101.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Topic Modeling with BERTopic

🔴 copied from the [Kubota Colab](https://colab.research.google.com/drive/1YsDp5_qGXGJKsEXsS8DO8CA_lqZc6EpA).  

`Topic Models` are methods to automatically organize a corpus of text into topics.

Topic Model process:
1. Data preparation
2. Tranform text to numeric vectors
3. Multidimensionality reduction
4. Clustering
5. Topic analysis
6. Cluster assignation


This notebook uses the library `BERTopic` which is a one-stop solution for topic modeling including handy functions for plotting and analysis. However, BERTopic does not have a function to extract the X and Y coords from UMAP. If we need the coordinates then use the notebooks `Topic_Models_using_Transformers` instead. In any other situation, when a quick analysis is needed this notebook may be better.

This notebook is also the one needed for the heatmap codes included in this folder.

`BERTopic` is Python library that handles steps 2 to 6.
BERT topic models use the transformer architechture to generate the embeds (i.e. the vector or numeric representation of words) and are currently the state-of-the-art method for vectorization.

This notebook shows how to use it.

---
Reading:
[Topic Modeling with Deep Learning Using Python BERTopic](https://medium.com/grabngoinfo/topic-modeling-with-deep-learning-using-python-bertopic-cf91f5676504)
[Advanced Topic Modeling with BERTopic](https://www.pinecone.io/learn/bertopic/)


# Requirements

## Packages installation and initialization

In [2]:
!pip install bertopic[visualization]

zsh:1: no matches found: bertopic[visualization]


In [1]:
import pandas as pd
import numpy as np
import time
import math
import uuid
import re
import os
import json
import pickle
from datetime import date
from itertools import compress
from bertopic import BERTopic
from umap import UMAP
from gensim.parsing.preprocessing import remove_stopwords
from sklearn.cluster import KMeans
from sentence_transformers import SentenceTransformer

/Users/cristian/Desktop/GitHub/clustering/env-tm/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Connect your Google Drive

In [2]:
from google.colab import drive
drive.mount('/content/drive')

ModuleNotFoundError: No module named 'google'

In [2]:
def find_e_keys(dictionary):
    # List comprehension to find keys starting with 'e'
    e_keys = [key for key in dictionary if str(key).lower().startswith('e')]
    return e_keys

# 🔴 Input files and options

Go to your Google Drive and create a folder in the root directory. We are going to save all related data in that directory.
Upload the dataset of news into the above folder.
- The dataset should be a `.csv` file.
- Every row in the dataset is a document
- It can any kind of columns. Some columns must contain the text we want to analyze. For example, a dataset of academic articles may contain a "Title" and/or "Abstract" column.

In [34]:
# The bibliometrics folder
# Colab
ROOT_FOLDER_PATH = "drive/MyDrive/Bibliometrics_Drive"

# Mac
ROOT_FOLDER_PATH = "/Users/cristian/Library/CloudStorage/GoogleDrive-cristianmejia00@gmail.com/My Drive/Bibliometrics_Drive"

# Change to the name of the folder where the dataset is uploaded inside the above folder
project_folder = 'Q324_EU_Act'

analysis_id = 'a01_tm__f01_e01__km01'

# Filtered label
settings_directive = "settings_analysis_directive_2025-01-30-17-06.json"

In [35]:
# Read settings
with open(f'{ROOT_FOLDER_PATH}/{project_folder}/{analysis_id}/{settings_directive}', 'r') as file:
    settings = json.load(file)

In [36]:
# Input dataset
dataset_file_path = f"{ROOT_FOLDER_PATH}/{settings['metadata']['project_folder']}/{settings['metadata']['filtered_folder']}/dataset_raw_cleaned.csv"

In [37]:
# Function to save files
def save_as_csv(df, save_name_without_extension, with_index):
    "usage: `save_as_csv(dataframe, 'filename')`"
    df.to_csv(f"{ROOT_FOLDER_PATH}/{save_name_without_extension}.csv", index=with_index)
    print("===\nSaved: ", f"{ROOT_FOLDER_PATH}/{save_name_without_extension}.csv")

In [38]:
# prompt: a function to save object to a pickle file
def save_object_as_pickle(obj, filename):
  """
  Saves an object as a pickle file.

  Args:
      obj: The object to be saved.
      filename: The filename of the pickle file.
  """
  with open(filename, "wb") as f:
    pickle.dump(obj, f)


In [39]:
# prompt: a function to load pickle object given a path
def load_pickle(path):
    with open(path, 'rb') as f:
        return pickle.load(f)


In [41]:
# Open the data file
df = pd.read_csv(f"{dataset_file_path}", encoding='latin-1')
print(df.shape)
df.head()

(1005, 16)


,X_N,uuid,WC,UT,PY,AU,TI,DT,PT,CR,C1,Z9,AB,Countries,IsoCountries,Institutions
0,1,e1e88a1c-5f8a-4c95-b6d7-d5fa007dc676,ArticleÊ1 - Subject matter`,id29929.7285477815,2024,EU,NaN,Article,Article,NaN,NaN,1,"""1. The purpose of this Regulation is to imp...",NaN,NaN,NaN
1,2,b4fedafe-238b-4bef-a1a3-398a23d4d9ab,ArticleÊ1 - Subject matter`,id372869.530597917,2024,EU,NaN,Article,Article,NaN,NaN,1,2. This Regulation lays down:,NaN,NaN,NaN
2,3,b51535a2-425d-4c49-9192-353bf3b3d7b2,ArticleÊ1 - Subject matter`,id692898.300406581,2024,EU,NaN,Article,Article,NaN,NaN,1,"""(a) harmonised rules for the placing on the ...",NaN,NaN,NaN
3,4,d1e2278d-15d5-4edb-ad15-0eea3d126df1,ArticleÊ1 - Subject matter`,id970016.957203564,2024,EU,NaN,Article,Article,NaN,NaN,1,(b) prohibitions of certain AI practices;,NaN,NaN,NaN
4,5,3a9f771e-2028-44cf-9c1e-6207a9ac1d99,ArticleÊ1 - Subject matter`,id872031.626035799,2024,EU,NaN,Article,Article,NaN,NaN,1,(c) specific requirements for high-risk AI sy...,NaN,NaN,NaN




---



## PART 2: Topic Model

In [42]:
# bibliometrics_folder
# project_folder
# project_name_suffix
# ROOT_FOLDER_PATH = f"drive/MyDrive/{bibliometrics_folder}"

#############################################################
# Embeddings folder
embeddings_folder_name = settings['tmo']['embeds_folder']

# Which column has the year of the documents?
my_year = settings['tmo']['year_column']

# Number of topics. Select the number of topics to extract.
# Choose 0, for automatic detection.
n_topics = settings['tmo']['n_topics']

# Minimum number of documents per topic
min_topic_size = settings['tmo']['min_topic_size']

In [43]:
# Get the embeddings back.
embeddings = load_pickle(f"{ROOT_FOLDER_PATH}/{project_folder}/{settings['metadata']['filtered_folder']}/{embeddings_folder_name}/embeddings.pck")
corpus =     pd.read_csv(f"{ROOT_FOLDER_PATH}/{project_folder}/{settings['metadata']['filtered_folder']}/{embeddings_folder_name}/corpus.csv").reset_index(drop=True)

In [44]:
# Combine embeddings
documents = corpus.text.to_list()

In [45]:
# corpus['uuid'] = [uuid.uuid4() for _ in range(len(corpus.index))]
# corpus['X_N'] = [i for i in range(1, len(corpus.index)+1)]

In [46]:
len(documents)

1005

In [47]:
#len(embeddings) == len(documents)
len(embeddings['embeddings']) == len(documents)

True

In [48]:
from hdbscan.hdbscan_ import HDBSCAN
# Execute the topic model.
# I suggest changing the values marked with #<---
# The others are the default values and they'll work fine in most cases.
# This will take several minutes to finish.

# Initiate UMAP
umap_model = UMAP(n_neighbors=15,
                  n_components=5,
                  min_dist=0.0,
                  metric='cosine',
                  random_state=100)

if n_topics == 0:
  # Initiate topic model with HDBScan (Automatic topic selection)
  topic_model_params = HDBSCAN(min_cluster_size=min_topic_size,
                               metric='euclidean',
                               cluster_selection_method='eom',
                               prediction_data=True)
else:
  # Initiate topic model with K-means (Manual topic selection)
  topic_model_params = KMeans(n_clusters = n_topics)

# Initiate BERTopic
topic_model = BERTopic(umap_model = umap_model,
                       hdbscan_model = topic_model_params,
                       min_topic_size=min_topic_size,
                       #nr_topics=15,          #<--- Footnote 1
                       n_gram_range=(1,3),
                       language='english',
                       calculate_probabilities=True,
                       verbose=True)



# Footnote 1: This controls the number of topics we want AFTER clustering.
# Add a hashtag at the beggining to use the number of topics returned by the topic model.
# When using HDBScan nr_topics will be obtained after orphans removal, and there is no warranty that `nr_topics < HDBScan topics`.
# thus, with HDBScan `nr_topics` means N topics OR LESS.
# When using KMeans nr_topics can be used to further reduce the number of topics.
# We use the topics as returned by the topic model. So we do not need to activate it here.

In [49]:
# Compute topic model
#topics, probabilities = topic_model.fit_transform(documents, embeddings)
topics, probabilities = topic_model.fit_transform(documents, embeddings['embeddings'])

2025-01-30 17:08:50,383 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2025-01-30 17:08:53,293 - BERTopic - Dimensionality - Completed ✓
2025-01-30 17:08:53,294 - BERTopic - Cluster - Start clustering the reduced embeddings
2025-01-30 17:08:53,313 - BERTopic - Cluster - Completed ✓
2025-01-30 17:08:53,320 - BERTopic - Representation - Extracting topics from clusters using representation models.
2025-01-30 17:08:53,480 - BERTopic - Representation - Completed ✓


In [50]:
# Get the list of topics
# Topic = the topic number. From the largest topic.
#         "-1" is the generic topic. Genericr keywords are aggegrated here.
# Count = Documents assigned to this topic
# Name = Top 4 words of the topic based on probability
# Representation = The list of words representing this topic
# Representative_Docs = Documents assigned to this topic
tm_summary = topic_model.get_topic_info()
tm_summary

,Topic,Count,Name,Representation,Representative_Docs
0,0,56,0_notified_bodies_body_assessment,"[notified, bodies, body, assessment, the, noti...",[The application for notification shall be acc...
1,1,52,1_surveillance_market surveillance_the_market,"[surveillance, market surveillance, the, marke...",[Any information or documentation obtained pur...
2,2,49,2_regulatory_and_ai regulatory_sandbox,"[regulatory, and, ai regulatory, sandbox, sand...",[Providers and prospective providers participa...
3,3,48,3_the_of_implementing_advisory forum,"[the, of, implementing, advisory forum, adviso...",[The Commission shall adopt implementing acts ...
4,4,46,4_in_highrisk ai_the_highrisk,"[in, highrisk ai, the, highrisk, conformity, o...",[For highrisk AI systems listed in point of A...
5,5,46,5_generalpurpose ai_generalpurpose_model_systemic,"[generalpurpose ai, generalpurpose, model, sys...",[The obligation set out in this Article shall ...
6,6,45,6_highrisk ai_highrisk_ai_and,"[highrisk ai, highrisk, ai, and, be, highrisk ...","[Providers of highrisk AI systems shall, The t..."
7,7,43,7_system_ai system_highrisk_highrisk ai system,"[system, ai system, highrisk, highrisk ai syst...",[Where the circumstances referred to in paragr...
8,8,42,8_the_authorised_mandate_the mandate,"[the, authorised, mandate, the mandate, to, au...",[The authorised representative shall terminate...
9,9,40,9_or_of_harm_the,"[or, of, harm, the, persons, safety, to, and, ...",[a the nature gravity and duration of the inf...


In [51]:
# Save the topic model assets
tm_folder_path = f'{ROOT_FOLDER_PATH}/{project_folder}/{settings["metadata"]["analysis_id"]}'

if not os.path.exists(tm_folder_path):
  !mkdir $tm_folder_path

tm_summary.to_csv(f'{tm_folder_path}/topic_model_info.csv', index=False)

In [52]:
# Number of topics found
found_topics = max(tm_summary.Topic) + 1
found_topics

33

In [53]:
# Confirm all documents are assigned
sum(tm_summary.Count) == len(corpus)

True

In [54]:
# Get top 10 terms for a topic
topic_model.get_topic(0)

[('notified', 0.032036358490834596),
 ('bodies', 0.02526655292427158),
 ('body', 0.022397913594335433),
 ('assessment', 0.02216894545082621),
 ('the', 0.021465507378238907),
 ('notified bodies', 0.019816530451538063),
 ('conformity assessment', 0.01948399065131862),
 ('notifying', 0.019097576037989415),
 ('conformity', 0.01742241433756636),
 ('shall', 0.016331490646483867)]

In [55]:
# Get the top 10 documents for a topic
topic_model.get_representative_docs(0)

['The application for notification shall be accompanied by a description of the conformity assessment activities the conformity assessment module or modules and the types of AI systems for which the conformity assessment body claims to be competent as well as by an accreditation certificate where one exists issued by a national accreditation body attesting that the conformity assessment body fulfils the requirements laid down in Article',
 'Where a notifying authority has sufficient reason to consider that a notified body no longer meets the requirements laid down in Article  or that it is failing to fulfil its obligations the notifying authority shall without delay investigate the matter with the utmost diligence In that context it shall inform the notified body concerned about the objections raised and give it the possibility to make its views known If the notifying authority comes to the conclusion that the notified body no longer meets the requirements laid down in Article  or that

In [56]:
# Others

# # Get the number of documents per topic (same as in the table above)
# topic_model.get_topic_freq(0)

# # Get the main keywords per topic
# topic_model.get_topics()

In [57]:
# Print the parameters used. (For reporting)
topic_model.get_params()

{'calculate_probabilities': True,
 'ctfidf_model': ClassTfidfTransformer(),
 'embedding_model': None,
 'hdbscan_model': KMeans(n_clusters=33),
 'language': 'english',
 'low_memory': False,
 'min_topic_size': 10,
 'n_gram_range': (1, 3),
 'nr_topics': None,
 'representation_model': None,
 'seed_topic_list': None,
 'top_n_words': 10,
 'umap_model': UMAP(angular_rp_forest=True, metric='cosine', min_dist=0.0, n_components=5, n_jobs=1, random_state=100, tqdm_kwds={'bar_format': '{desc}: {percentage:3.0f}%| {bar} {n_fmt}/{total_fmt} [{elapsed}]', 'desc': 'Epochs completed', 'disable': True}),
 'vectorizer_model': CountVectorizer(ngram_range=(1, 3)),
 'verbose': True,
 'zeroshot_min_similarity': 0.7,
 'zeroshot_topic_list': None}

In [58]:
tm_params = dict(topic_model.get_params())
for key, value in tm_params.items():
    tm_params[key]=  str(value)
with open(f'{tm_folder_path}/topic_model_params.json', 'w') as f:
    json.dump(tm_params, f, ensure_ascii=False, indent=4)
    print('Done')

Done


In [59]:
# Get the topic score for each paper and its assigned topic
topic_distr, _ = topic_model.approximate_distribution(documents, batch_size=1000)
distributions = [distr[topic] if topic != -1 else 0 for topic, distr in zip(topics, topic_distr)]

100%|██████████| 2/2 [00:00<00:00,  9.62it/s]


In [60]:
topic_distr

array([[0.        , 0.02962026, 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.0179666 , 0.        ,
        0.        ],
       ...,
       [0.        , 0.23717634, 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.01578444, 0.09248167, 0.        , ..., 0.03457878, 0.01226191,
        0.01897028]])

In [61]:
# Document information. Including the topic assignation
dataset_clustering_results = topic_model.get_document_info(documents, df = corpus, metadata={"Score": distributions})

# Standar format for report analysis
dataset_clustering_results = dataset_clustering_results.drop(columns=['text'])
dataset_clustering_results['X_E'] = dataset_clustering_results['Score']
dataset_clustering_results['X_C'] = dataset_clustering_results['Topic'] + 1
dataset_clustering_results['level0'] = dataset_clustering_results['Topic'] + 1
dataset_clustering_results['cl99'] = False
dataset_clustering_results['cl-99'] = False
dataset_clustering_results.head()

,UT,Document,Topic,Name,Representation,Representative_Docs,Top_n_words,Representative_document,Score,X_E,X_C,level0,cl99,cl-99
0,id29929.7285477815,The purpose of this Regulation is to improve t...,9,9_or_of_harm_the,"[or, of, harm, the, persons, safety, to, and, ...",[a the nature gravity and duration of the inf...,or - of - harm - the - persons - safety - to -...,False,0.101295,0.101295,10,10,False,False
1,id372869.530597917,This Regulation lays down,14,14_regulation_this_this regulation_union,"[regulation, this, this regulation, union, of,...",[The data listed in Section C of Annex VIII sh...,regulation - this - this regulation - union - ...,False,1.000000,1.000000,15,15,False,False
2,id692898.300406581,a harmonised rules for the placing on the mar...,18,18_or_ai_the_of,"[or, ai, the, of, of ai, on, person, ai system...",[d the placing on the market the putting into...,or - ai - the - of - of ai - on - person - ai ...,False,0.543130,0.543130,19,19,False,False
3,id970016.957203564,b prohibitions of certain AI practices,13,13_regulation_this regulation_apply_or,"[regulation, this regulation, apply, or, ai, t...",[This Regulation does not apply to AI systems ...,regulation - this regulation - apply - or - ai...,False,0.000000,0.000000,14,14,False,False
4,id872031.626035799,c specific requirements for highrisk AI syste...,6,6_highrisk ai_highrisk_ai_and,"[highrisk ai, highrisk, ai, and, be, highrisk ...","[Providers of highrisk AI systems shall, The t...",highrisk ai - highrisk - ai - and - be - highr...,False,0.250876,0.250876,7,7,False,False


In [62]:
# Save the dataframe
dataset_clustering_results.to_csv(f'{tm_folder_path}/dataset_minimal.csv', index=False)

In [63]:
# Save the topic model
topic_model.save(f'{tm_folder_path}/topic_model_object.pck')

2025-01-30 17:08:54,244 - BERTopic - WARNING: When you use `pickle` to save/load a BERTopic model,please make sure that the environments in which you saveand load the model are **exactly** the same. The version of BERTopic,its dependencies, and python need to remain the same.




---

